# Análisis de tópicos

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Ciencia de Datos

**Trabajo práctico desarrollado por: [Olivia Gutú](https://oliviagutu.github.io/TEORIA-DE-LA-COMPUTACION/) y [Julio Waissman](http://mat.uson.mx/~juliowaissman/)** 



En esta libreta vamos a experimentar con el análisis de tópicos a partir de un corpus en español muy interesante para realizar análisis de tópicos: Una colección de sonetos del siglo de oro español. El córpus incluye mucha información adicional (metadatos) como título, autor, y sobre todo la notación métrica del soneto. Sin embargo para este trabajo no vamos a considerar ninguna de esta información adicional.

El corpus se encuentra [en este proyecto de github](https://github.com/bncolorado/CorpusSonetosSigloDeOro) y para su uso académico lo debemos citar correctamente en la referencia:

> Navarro-Colorado, Borja; Ribes Lafoz, María, and Sánchez, Noelia (2015) "Metrical annotation of a large corpus of Spanish sonnets: representation, scansion and evaluation" 10th edition of the Language Resources and Evaluation Conference 2016 Portorož, Slovenia.[PDF](http://www.dlsi.ua.es/%7Eborja/navarro2016_MetricalPatternsBank.pdf)


## 1. Obtención del *corpus*

Para obtener el *corpus* es necesario clonar el proyecto original. Desde el punto de mentaje (esto es, la carpeta `curso-pln` centro del contenedor (aunque tambien puede ser por fuera), se usa el comando:

```
git clone https://github.com/bncolorado/CorpusSonetosSigloDeOro.git
```

Una vez clonado, es necesario extraer la información de los archivos `xml`. La generación del *corpus* sin normalizar ya la dejo desarollada.

In [ ]:
import xml.etree.ElementTree as et # Para manejar buffers con estructura xml
import os

# Si guardaste los sonetos en otra dirección, 
# aqui es donde debes de cambiarlos
archivos_path = "./CorpusSonetosSigloDeOro/"

def lee_soneto(archivo):
    soneto = ""
    arbol = et.parse(archivo)
    raiz = arbol.getroot()
    for poema in raiz.find('{http://www.tei-c.org/ns/1.0}text'):
        if poema.tag == '{http://www.tei-c.org/ns/1.0}body':
            for parrafo in poema:
                if parrafo.tag == '{http://www.tei-c.org/ns/1.0}lg':
                    for linea in parrafo:
                        soneto += (linea.text + '\n')
                    soneto += '\n'
    return soneto

corpus_sonetos = []
for (dirpath, dirnames, filenames) in os.walk(archivos_path):
    if not dirnames:
        for filename in filenames:
            if filename[-4:] == '.xml':
                corpus_sonetos.append(lee_soneto(dirpath + '/' + filename))
            
len(corpus_sonetos)

Ejecuta varias veces la celda de abajo para ver en forma aleatoria si los sonetos se descargaron correcatmente

In [ ]:
import random
print(corpus_sonetos[random.randint(0, 5077)])

## 2. Procesaiento de texto

Ahora es necesario normalizar el texto con fin de utilizarlo en modelado de tópicos.

El texto lo vamos a tratar con al menos los siguientes requisitos:

1. Utilizar todas las palabras en minúsculas
1. Eliminar los signos de puntuación
2. Eliminar palabras vacias
3. Eliminar las palabras que no aportan significados en la poesía. 
    
Realiza esta normalización utilizando el módulo *spacy* con el modelo `es_core_news_sm`, pero sientete con la libertad de cargar el modelo 
`es_core_news_md` o inclusive el modelo `es_core_news_lg`. 

El corpus procesado (*normalizado* también se le dice) se guardará en una lista de textos que llamaremos `corpus_tratado`

In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [ ]:
def procesa_poema(doc): 
    
    return [
        token.norm_ for token in doc
        if token.is_alpha and not token.like_num and not token.is_stop and
           token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']
    ]

corpus_tratado = [procesa_poema(doc) for doc in nlp.pipe(corpus_sonetos)]


Y utiliza esta celda para probar si se normalizaron vien los sontetos

In [ ]:
i = random.randint(0, 5077)
print(i)
print(corpus_sonetos[i])
print("********************")
print(corpus_tratado[i])


## 3. Modelado de tópicos con LDA

Ahora, ya con el corpus tratado, desarrolla tu modelo LDA en *gensim*, siguiendo los siguientes pasos:

1. Crea un diccionario con el corpus tratado, donde elimines a todas las
   palabras que no aparezcan en al menos `min_df` documentos, y las palabras que 
   aparezcan el más del `max_df` porciento de documentos (por default `min_df = 5`
   y `max_df = 0.3`).
2. Genera un corpus listo para su uso, aplicando el metodo de bolsa de palabras.
3. Aplica el método para modelar con LDA. Establece el número de iteraciones para
   el método de estimación así como el numero de pasos que realiza el algoritmo
   de optimización iterativa (por default 100 y 5, respectivamente).
   
Guarda el modelo en la variable `modelo_lda` (opcionalmente puedes guardar el modelo si lo quieres usar más adelante)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from gensim import models, corpora

# Genera el diccionario de palabras
diccionario = corpora.Dictionary(corpus_tratado)
diccionario.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus = [diccionario.doc2bow(doc) for doc in corpus_tratado]

Y ahora hacemos el modelado de tópicos con LDA

In [ ]:
# Número de tópicos
n_topicos = 10

# Genera el modelo LDA
modelo_lda = models.LdaModel(corpus=corpus, num_topics=n_topicos, id2word=diccionario, iterations=100, passes=10)

# Si quieres guardar el modelo se hace esto
#modelo_lda.save("sonetosLDA.model")

Y podemos usar otros métodos como el de *Hierachical Dirichlet Process*

In [ ]:
modelo_hdp = models.hdpmodel.HdpModel(corpus, diccionario, T=10)

Y ahora veamos los modelos como se definen por sus primeras 5 palabras clave 

In [ ]:
for (i, topico) in modelo_lda.print_topics(num_topics=n_topicos, num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

In [ ]:
for (i, topico) in modelo_hdp.print_topics(num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

Y ahora veamos como se clasifica un documento utilizando tanto LDA como HDP

In [ ]:
i = random.randint(0, 5077)

poema = corpus_sonetos[i]
ids = diccionario.doc2bow(corpus_tratado[i])

print("El soneto: \n\n" + poema)

topicos_lda = modelo_lda[ids]
topicos_lda.sort(key=lambda x:x[1], reverse=True)

topicos_hdp = modelo_hdp[ids]
topicos_hdp.sort(key=lambda x:x[1], reverse=True)

print("Pertenece a los tópicos (con el modelo LDA)")
for (topico, peso) in topicos_lda:
    print("\tTopico {} con un peso de {}".format(topico, peso))

print("Pertenece a los tópicos (con el modelo HDP)")
for (topico, peso) in topicos_hdp:
    print("\tTopico {} con un peso de {}".format(topico, peso))

Tambien se puede obtener la lista de palabras que describen a cada tópico

Mejor aún, utiliza *pyLDAvis* para visualizar y analizar los tópicos desarrollados.

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(modelo_lda, corpus, diccionario)
pyLDAvis.display(vis_data)

In [ ]:
vis_data = gensimvis.prepare(modelo_hdp, corpus, diccionario)
pyLDAvis.display(vis_data)

## 4 Evaluando modelos

Para evaluar un modelo, la forma más sencilla es utilizando el modelo para evaluar la perplejidad del corpus

In [ ]:
print(f"El logaritomo de la perplejidad del corpus al modelo LDA es {modelo_lda.log_perplexity(corpus)}")

Otro método es analizando la [coherencia de los tópicos](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf), ponemos algunos resultados:

In [ ]:
cm_lda = models.CoherenceModel(model=modelo_lda, texts=corpus_tratado, coherence='c_v')
cm_hdp = models.CoherenceModel(model=modelo_hdp, texts=corpus_tratado, coherence='c_v')


print("Coherencia del modelo LDA (C_V): {}".format(cm_lda.get_coherence()))
for (topico, coherencia) in enumerate(cm.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

print("\n\nCoherencia del modelo HDP (C_V): {}".format(cm_hdp.get_coherence()))
for (topico, coherencia) in enumerate(cm.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

Con la coherencia se puede obtener el conjunto de palabras que mejor definen cada tópico

In [ ]:
print("Las palabra que definen los tópicos en LDA\n")
t_palabras = cm_lda.top_topics_as_word_lists(model=modelo_lda, dictionary=diccionario)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")
    
print("\n\nLas palabra que definen los tópicos en HDP\n")
t_palabras = cm_hdp.top_topics_as_word_lists(model=modelo_hdp, dictionary=diccionario)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")

## 5 Responde a las preguntas siguientes

Cada pregunta implica experimentación y reflexión, por favor extiende lo que consideres necesario tus explicaciones:

##### 1. ¿Cual es el número de tópicos que consideras te ofrece una mejor separación entre ellos, y mejor significado (dejando al resto de los parámetros en valores por default)? ¿Porqué? ¿Algunos tópicos son redundantes entre si? ¿Podrías asignarle un nombre a cada tópico?

_ingresa aquí tu respuesta_

##### 2. ¿Que pasa si eliminas los pronombres personales en la normalización del texto? ¿Y si dejas solo los verbos? ¿Y si eliminamos los verbos? ¿Que pasa con el modelo large de SpaCy? ¿Que pasa si lematizas? ¿Hay alguna modificación al preprocesamiento de los documentos que implique una mejor distribución de tópicos (con el resto de los valores por default y el número de tópicos que seleccionaste en la pregunta 1)?

_ingresa aquí tu respuesta_

##### 3. ¿Que pasa si reduces el número de iteraciones en la inferencia a 50? ¿Que pasa si la aumentas a 200? ¿Y si reduces el numero de iteraciones del algoritmo de optimización a solo 1? ¿Si lo aumentas a 100 (ten cuidado, puede tardar mucho)? ¿Consideras que estos parámetros tienen mucha influencia en el resultado del método de LDA?

_ingresa aquí tu respuesta_

##### 4. ¿Que pasa si aceptas todas las palabras que aparezcan en menos del 80% de los documentos? ¿Que pasaría si limitas el vocabulario a las palabras que solamente aparezcan en el $\frac{1.5}{\text(numero\ de\ tópicos)}$ por ciento de los documentos? ¿Cual es el equilibrio que consideras que es el más acertado? 

_ingresa aquí tu respuesta_

##### 5. ¿Qué modelo te parece el mejor, después de probar con diferentes cosas?

Si tienes más de un modelo que te parece interesante, puedes utilizar los métodos de medición de coherencia de los tópicos para tomar una desición.

_ingresa aquí tu respuesta_